### 1.0 Imports

In [45]:
import os
import json
import requests
import logging
import configparser
import random
from typing import Any
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, BaseTool
from langchain.llms import Ollama
from langchain_community.tools import DuckDuckGoSearchRun
#from thehive4py import TheHiveApi
#from thehive4py.api import TheHiveApi
#from thehive4py.models import *
#from thehive4py.query import *



### 2.0 Configurations

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

search_tool = DuckDuckGoSearchRun()
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
# os.environ["SERPER_API_KEY"] = "Your Key" # serper.dev API key

# OPENAI_API_BASE='http://localhost:11434/v1'
# OPENAI_MODEL_NAME='phi'  # Adjust based on available model
# OPENAI_API_KEY=''

# Define the local LLM to be used
ollama_openhermes = Ollama(model="openhermes")

# You can choose to use a local model through Ollama for example. See https://docs.crewai.com/how-to/LLM-Connections/ for more information.

# search_tool = SerperDevTool()

# Connection to TheHive5 using thehive4py
#THEHIVE_URL = 'http://192.168.91.1:9000'
#THEHIVE_API_KEY = 'FKzlMouE6NkGHL8oQmLRbFyX7oIPccnj'

#api = TheHiveApi(THEHIVE_URL, THEHIVE_API_KEY)

#query = And(
#    Eq('tlp', Tlp.AMBER.value),
#    Eq('severity', Severity.HIGH.value)
    #Like('title', '*MALSPAM*')
#)
#response = api.find_alerts(query=query)

# Print the JSON response 
#print(json.dumps(response.json(), indent=4, sort_keys=True))

#thehive_api = TheHiveApi(
#        url="http://192.168.91.1:9000",
#        apikey="FKzlMouE6NkGHL8oQmLRbFyX7oIPccnj",
#    )

### 3.0 TheHive Connection and Functions

In [52]:
class TheHive:
    class Config:
        arbitrary_types_allowed = True
        
    def __init__(self):
        config = configparser.ConfigParser()
        config.read('config.ini')
        # Read TheHive instance URL from the config file
        self.thehive_url = config['DEFAULT']['thehive_url']
        # Read TheHive API key from the config file
        self.thehive_api_key = config['DEFAULT']['thehive_api_key']
        
    def _validate_api_key(self):
        if len(self.thehive_api_key) < 20:
            raise ValueError("TheHive API key seems too short. Please check your configuration.")
        
    # Adjust the query parameters according to TheHive's API documentation
    def query_alerts(self, severity=2):
        if severity not in [1, 2, 3]:
            raise ValueError("Invalid severity level. Allowed values: 1 (low), 2 (medium), 3 (high)")

        query_params = {
            "query": [
                {
                    "_name": "listAlert"
                },
                {
                    "_name": "filter",
                    "_field": "severity",
                    "_value": severity
                },
                {
                    "_name": "sort",
                    "_fields": [
                        {
                            "date": "desc"
                        }
                    ]
                },
                {
                    "_name": "page",
                    "from": 0,
                    "to": 15,
                    "extraData": [
                        "importDate",
                        "caseNumber"
                    ]
                }
            ]
        }

        try:
            response = requests.post(
                f'{self.thehive_url}/v1/query?name=alerts',
                json=query_params,
                headers={'Authorization': f'Bearer {self.thehive_api_key}'}
            )
            response.raise_for_status()  # Raise for non-200 status codes

            alerts = json.loads(response.text)
            print(json.dumps(alerts, indent=4))

        except requests.exceptions.HTTPError as err:
            logging.error(f"HTTP Error: {err}")
        except requests.exceptions.Timeout as err:
            logging.error(f"Request timed out: {err}")
        except requests.exceptions.RequestException as err:
            logging.error(f"General TheHive request error: {err}")

    def create_alert(self, title, description, severity=3, observables=None):
        # Add more extensive input validation here... 

        alert = {
            'title': title,
            'type': 'AI Agents',
            'source': 'CrewAI',
            'sourceRef': str(random.randint(1, 1000000)),  # Generate a random number between 1 and 1,000,000
            'description': description,
            'severity': severity
        }
        
        #if observables:
        #    alert['observables'] = observables
            
        files = {}
        if observables:
            alert['observables'] = observables
            for observable in observables:
                if 'attachment' in observable:
                    files[observable['attachment']] = open(observable['attachment'], 'rb')

        while True:
            try:
                response = requests.post(
                    f'{self.thehive_url}/v1/alert',
                    files={
                        '_json': json.dumps(alert),
                        **files
                    },
                    headers={'Authorization': f'Bearer {self.thehive_api_key}'}
                )
                response.raise_for_status()
                break  # If the request was successful, break the loop

            except requests.exceptions.HTTPError as err:
                error_message = json.loads(response.text).get('message', '')
                if 'already exists' in error_message:
                    # If the alert already exists, generate a new random number and retry
                    alert['sourceRef'] = str(random.randint(1, 1000000))
                else:
                    # If the error is not because the alert already exists, log the error and break the loop
                    logging.error(f"HTTP Error: {err}")
                    break

            except requests.exceptions.RequestException as err:
                logging.error(f"General TheHive request error: {err}")
                break

In [22]:
if __name__ == "__main__":
    thehive = TheHive()
    thehive._validate_api_key()  # Check API key on initialization

new_observables = [
    {'dataType': 'ip', 'value': '1.1.1.1'},
    {'dataType': 'url', 'value': 'http://example.com'}
    ]
thehive.create_alert("Test from CrewAI Tool", "Create an alert with the API call to verify that the tool is working as expected", observables=new_observables)

In [47]:
if __name__ == "__main__":
    thehive = TheHive()
    thehive._validate_api_key()  # Check API key on initialization

    # Example usage
    thehive.query_alerts(severity=2)

#    new_observables = [
#        {'dataType': 'ip', 'value': '192.168.1.1'},
#        {'dataType': 'url', 'value': 'http://example.com'},
#    ]
#    thehive.create_alert("Test from CrewAI", "Description...", observables=new_observables)

[
    {
        "_id": "~4047072",
        "_type": "Alert",
        "_createdBy": "am@onzo.local",
        "_updatedBy": "am@onzo.local",
        "_createdAt": 1712653962310,
        "_updatedAt": 1712653962353,
        "type": "wazuh_alert",
        "source": "wazuh",
        "sourceRef": "96ca83",
        "title": "CVE-2024-2511 affects openssl",
        "description": "### Timestamp\n| key | val |\n| ------ | ------ |\n| **timestamp** | 2024-04-09T09:12:39.528+0000 |\n### Rule\n| key | val |\n| ------ | ------ |\n| **rule.level** | 7 |\n| **rule.description** | CVE-2024-2511 affects openssl |\n| **rule.id** | 23504 |\n| **rule.firedtimes** | 2 |\n| **rule.mail** | False |\n| **rule.groups** | ['vulnerability-detector'] |\n| **rule.gdpr** | ['IV_35.7.d'] |\n| **rule.pci_dss** | ['11.2.1', '11.2.3'] |\n| **rule.tsc** | ['CC7.1', 'CC7.2'] |\n### Agent\n| key | val |\n| ------ | ------ |\n| **agent.id** | 000 |\n| **agent.name** | wazuh.manager |\n| **agent.ip** | 127.0.0.1 |\n### Mana

### 4.0 Tools

In [48]:
class TheHiveToolOLD(BaseTool):
    name: str = "TheHiveTool"
    description: str = "Enables reading and extracting data from TheHive5 Case management platform."
    thehive_instance: TheHive = None  # Add this line
    
    class Config:
        arbitrary_types_allowed = True
        
    def __init__(self):
        super().__init__()  # Add this line
        config = configparser.ConfigParser()
        config.read('config.ini')
        self.thehive_instance = TheHive()

    def _run(self, action_kwargs: dict) -> None:
        action = action_kwargs.get('action')
        kwargs = action_kwargs.get('kwargs', {})
        try:
            if action == "query_alerts":
                self.thehive_instance.query_alerts(**kwargs) 
            elif action == "create_alert":
                self.thehive_instance.create_alert(**kwargs)
            else:
                raise ValueError(f"Unsupported action: {action}")
        except Exception as e:
            logging.error(f"Error during TheHive operation: {e}")

In [49]:
class TheHiveTool(BaseTool):
    name: str = "TheHiveTool"
    description: str = "Enables reading and extracting data from TheHive5 Case management platform."
    thehive_instance: TheHive = None  # Add this line
    
    class Config:
        arbitrary_types_allowed = True
        
    def __init__(self):
        super().__init__()  # Add this line
        config = configparser.ConfigParser()
        config.read('config.ini')
        self.thehive_instance = TheHive()

    def _run(self, action_kwargs: dict = None, **kwargs) -> None:
        if action_kwargs is not None:
            action = action_kwargs.get('action')
            kwargs = action_kwargs.get('kwargs', {})
        else:
            action = kwargs.pop('action', None)
        try:
            if action == "query_alerts":
                self.thehive_instance.query_alerts(**kwargs) 
            elif action == "create_alert":
                self.thehive_instance.create_alert(**kwargs)
            else:
                raise ValueError(f"Unsupported action: {action}")
        except Exception as e:
            logging.error(f"Error during TheHive operation: {e}")


In [50]:
if __name__ == "__main__":
    tool = TheHiveTool()

    # Example: Query alerts with severity 'high'
    #tool._run("query_alerts", severity="high")
    tool._run({"action": "query_alerts", "kwargs": {"severity": 3}})

[
    {
        "_id": "~8163440",
        "_type": "Alert",
        "_createdBy": "am@onzo.local",
        "_createdAt": 1712668671441,
        "type": "test",
        "source": "CrewAI",
        "sourceRef": "1",
        "title": "Test from CrewAI Tool",
        "description": "Create an alert with the API call to verify that the tool is working as expected",
        "severity": 3,
        "severityLabel": "HIGH",
        "date": 1712668671432,
        "tags": [],
        "tlp": 2,
        "tlpLabel": "AMBER",
        "pap": 2,
        "papLabel": "AMBER",
        "follow": true,
        "customFields": [],
        "observableCount": 0,
        "status": "New",
        "stage": "New",
        "extraData": {
            "importDate": null,
            "caseNumber": null
        },
        "newDate": 1712668671435,
        "timeToDetect": 0
    },
    {
        "_id": "~8569048",
        "_type": "Alert",
        "_createdBy": "am@onzo.local",
        "_createdAt": 1712316756734,
     

In [51]:
if __name__ == "__main__":
    tool = TheHiveTool()
    
    # Example: Create an alert
    tool._run(
        action="create_alert",
        title="CrewAI Tool Alert",
        description="Create an alert with the API call to verify that the tool is working as expected",
        severity=3,
        observables=[
            {'dataType': 'ip', 'value': '8.8.8.8'},
            {'dataType': 'url', 'value': 'http://example.com'}
        ]
    )


In [53]:
if __name__ == "__main__":
    tool = TheHiveTool()
    
    
new_observables = [
    {'dataType': 'ip', 'value': '1.1.1.1'},
    {'dataType': 'url', 'value': 'http://www.cloudflare.com'}
    ]
    
    # Example: Create an alert
tool._run(
        {
            "action": "create_alert",
            "kwargs": {
                "title": "CrewAI Tool Alert",
                "description": "Create an alert with the API call to verify that the tool is working as expected",
                "severity": 3,
                "observables": new_observables
            }
        }
    )
